In [7]:
from dgp import *

# 1. Simulate or load data
sample_size = 800
min_degree = 8
max_degree = 10
adj = get_graph(sample_size, min_degree, max_degree, seed=2)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=30,
    burnin_R=10, seed=1)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 40/40 [00:01<00:00, 30.61it/s]


In [2]:
n_cpu = 10

In [8]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=200,
    burnin_R=50,
    seed=0
)

{'average': 0.14491875,
 'direct_effect': -0.10791875,
 'spillover_effect': -0.458925,
 'psi_1_gamma': 0.116075,
 'psi_0_gamma': 0.22399375,
 'psi_zero': 0.68291875}

In [9]:
from drnet import doubly_robust
from drnet_fast import doubly_robust_optimized

In [10]:
doubly_robust(A_chain[-1], L_chain[-1], Y_chain[-1], adj)

{'average': 0.2257484981840708,
 'direct_effect': -0.8846821493573541,
 'spillover_effect': 0.33693760343068935,
 'psi_1_gamma': 0.08887317101310464,
 'psi_0_gamma': 0.9735553203704588,
 'psi_zero': 0.6366177169397694}

In [11]:
doubly_robust_optimized(A_chain[-1], L_chain[-1], Y_chain[-1], adj)

(800, 1000) (800, 1000) (800, 1000) (800,) (800, 9, 1000)
(800, 1000) (800, 1000) (800, 1000) (800,) (800, 9, 1000)
(800, 1000) (800, 1000) (800, 1000) (800,) (800, 9, 1000)


{'average': 0.2257484981840708,
 'direct_effect': -0.8846821493573539,
 'spillover_effect': 0.3369376034306891,
 'psi_1_gamma': 0.08887317101310464,
 'psi_0_gamma': 0.9735553203704586,
 'psi_zero': 0.6366177169397694}

In [6]:
np.zeros((800, 1000, 9)).transpose(0, 2, 1).shape

(800, 9, 1000)

In [2]:
from drnet import *

def doubly_robust_debug(A, L, Y, adj_matrix, treatment_allocation=0.7, num_rep=1000, seed=1):
    np.random.seed(seed)
    
    # fit models
    X_y = build_design_matrix_Y(A, L, Y, adj_matrix)
    model_y = fit_logistic_model(X_y, Y)
    X_a = build_design_matrix_A(L, A, adj_matrix)
    model_a = fit_logistic_model(X_a, A)
    
    # compute pi
    gamma = np.concatenate([model_a.intercept_, model_a.coef_.flatten()])
    N = adj_matrix.shape[0]
    neighbours = [list(adj_matrix[i].nonzero()[0]) for i in range(N)]
    L_nb = get_neighbor_summary(L, adj_matrix)
    GL = gamma[0] + L.dot(np.array([gamma[1], gamma[3], gamma[5]])) \
        + L_nb.dot(np.array([gamma[2], gamma[4], gamma[6]]))
        
    denominator = get_norm_constant(A, GL, neighbours, gamma, adj_matrix)
    print(denominator)
    """
    # compute the influence function
    a_mat = np.random.binomial(1, treatment_allocation, size=(Y.shape[0], num_rep))
    numerator_vec, I = get_numerator_pi_vec(a_mat, A, GL, neighbours, gamma, adj_matrix, Atype='all')
    pi_vec = numerator_vec / denominator[:, None]
    
    psi_gamma = []
    for i in range(num_rep):
        X_y_eval = build_design_matrix_Y(a_mat[:,i], L, Y, adj_matrix)
        beta_hat = compute_beta_probs(X_y_eval, model_y, Atype='all')
        psi = beta_hat + I[:,i] / pi_vec[:, i] * (Y - beta_hat)
        psi_gamma.append(psi.mean())
    
    numerator, I = get_numerator_pi_vec(a_mat, A, GL, neighbours, gamma, adj_matrix, Atype='ind_treat_1')
    pi_1_vec = numerator / denominator[:, None]
    psi_1_gamma = []
    for i in range(num_rep):
        X_y_eval = build_design_matrix_Y(a_mat[:,i], L, Y, adj_matrix)
        beta_hat = compute_beta_probs(X_y_eval, model_y, Atype='ind_treat_1')
        psi = beta_hat + I[:,i] / pi_1_vec[:, i] * (Y - beta_hat)
        psi_1_gamma.append(psi.mean())
        
    numerator, I = get_numerator_pi_vec(a_mat, A, GL, neighbours, gamma, adj_matrix, Atype='ind_treat_0')
    pi_0_vec = numerator / denominator[:, None]
    psi_0_gamma = []
    for i in range(num_rep):
        X_y_eval = build_design_matrix_Y(a_mat[:,i], L, Y, adj_matrix)
        beta_hat = compute_beta_probs(X_y_eval, model_y, Atype='ind_treat_0')
        psi = beta_hat + I[:,i] / pi_0_vec[:, i] * (Y - beta_hat) 
        psi_0_gamma.append(psi.mean())
    
    a_mat = np.zeros((Y.shape[0],1))
    numerator, I = get_numerator_pi_vec(a_mat, A, GL, neighbours, gamma, adj_matrix, Atype='all_0')
    pi_zero_vec = numerator / denominator[:, None]
    psi_zero = []
    X_y_eval = build_design_matrix_Y(a_mat, L, Y, adj_matrix)
    beta_hat = compute_beta_probs(X_y_eval, model_y, Atype='all_0')
    psi =  beta_hat + I[:,0] / pi_zero_vec[:, 0] * (Y - beta_hat)
    psi_zero.append(psi.mean())
    
    # Compute effects
    avg_psi_gamma = np.mean(psi_gamma)
    direct_effect = np.mean(psi_1_gamma) - np.mean(psi_0_gamma)
    spillover_effect = np.mean(psi_0_gamma) - np.mean(psi_zero)
    
    return {
        "average": avg_psi_gamma,
        "direct_effect": direct_effect,
        "spillover_effect": spillover_effect,
        "psi_1_gamma": np.mean(psi_1_gamma),
        "psi_0_gamma": np.mean(psi_0_gamma),
        "psi_zero": np.mean(psi_zero),
    }
    """

doubly_robust_debug(A_chain[-1], L_chain[-1], Y_chain[-1], adj)

[2.00776731 2.76346505 2.76901912 2.27685091 2.93332973 2.88434615
 3.17410733 3.0564813  2.33675073 2.29967421 2.96666132 4.70477878
 4.46501719 2.11032003 2.11359901 2.09580888 2.21655518 3.17602906
 2.76517791 3.92189041 2.26742055 2.25793069 2.03919625 2.18647255
 3.10544468 2.71561201 4.42362918 2.06885728 2.02863728 3.03523457
 1.92678242 2.22622765 3.06224937 2.7609939  2.67533606 3.02106674
 2.13097052 3.31612629 2.04671669 2.18451655 1.98587915 2.18314757
 2.66974176 1.93863547 2.17674661 3.06612764 2.24959643 2.98904133
 4.40006458 2.12940061 2.06418382 2.65930578 1.93863547 3.13143522
 3.18569219 2.35660795 2.27795953 3.01302729 4.39885832 2.10794522
 3.07814378 2.00651298 2.0536207  4.01755691 3.02359874 2.67721363
 2.33675073 2.98089408 2.02101112 2.1720401  2.13955396 2.98089408
 4.38138953 2.97910338 2.98899211 2.35253006 2.07167746 2.79827196
 3.0203904  2.78050783 1.98720054 3.18982673 2.25565514 3.01760173
 3.06314903 2.65931095 2.19396241 2.20249144 2.2162998  2.2748

In [2]:
from drnet_fast import *

def doubly_robust_optimized_debug(A, L, Y, adj_matrix, treatment_allocation=0.7, num_rep=1000, seed=1):
    np.random.seed(seed)
    N = len(Y)
    neighbours = [list(adj_matrix[i].nonzero()[0]) for i in range(N)]

    # Fit outcome and treatment models
    X_y = build_design_matrix_Y(A, L, Y, adj_matrix)
    model_y = fit_logistic_model(X_y, Y)
    X_a = build_design_matrix_A(L, A, adj_matrix)
    model_a = fit_logistic_model(X_a, A)
    gamma = np.concatenate([model_a.intercept_, model_a.coef_.flatten()])

    # Compute GL and reuse across tasks
    L_nb = get_neighbor_summary(L, adj_matrix)
    GL = gamma[0] + L.dot(np.array([gamma[1], gamma[3], gamma[5]])) + \
         L_nb.dot(np.array([gamma[2], gamma[4], gamma[6]]))

    # Compute normalization constant once
    denominator = get_norm_constant_fast(A, GL, neighbours, gamma, adj_matrix, n_rep=num_rep)
    print(denominator)

    # Sample treatment matrices
    a_mat = np.random.binomial(1, treatment_allocation, size=(N, num_rep))

    # Precompute shared covariate design components
    L, L_nb, Y_nb = build_design_matrix_Y_shared(L, Y, adj_matrix)

    def estimate_effect(Atype):
        numerator_vec, I = get_numerator_pi_vec_fast(a_mat, A, GL, neighbours, gamma, adj_matrix, Atype=Atype)
        pi_vec = numerator_vec / denominator[:, None]
        X_y_eval_batch = build_design_matrix_Y_batch(a_mat, L, L_nb, Y_nb, adj_matrix)
        beta_hat_batch = compute_beta_probs_batch(X_y_eval_batch, model_y, Atype=Atype)  # shape (N, num_rep)
        psi_batch = beta_hat_batch + I / pi_vec * (Y[:, None] - beta_hat_batch)
        return psi_batch.mean()

    # Estimate all required effects
    avg_psi_gamma = estimate_effect('all')
    psi_1_gamma = estimate_effect('ind_treat_1')
    psi_0_gamma = estimate_effect('ind_treat_0')

    # Special case for psi_zero with all-0 treatment vector
    a_zero = np.zeros((N, 1), dtype=int)
    numerator, I = get_numerator_pi_vec_fast(a_zero, A, GL, neighbours, gamma, adj_matrix, Atype='all_0')
    pi_zero_vec = numerator / denominator[:, None]
    X_y_eval = build_design_matrix_Y_batch(a_zero, L, L_nb, Y_nb, adj_matrix)[:, :, 0]
    beta_hat = model_y.predict_proba(X_y_eval)[:, 1]
    psi = beta_hat + I[:, 0] / pi_zero_vec[:, 0] * (Y - beta_hat)
    psi_zero = psi.mean()

    # Compute effects
    direct_effect = psi_1_gamma - psi_0_gamma
    spillover_effect = psi_0_gamma - psi_zero

    return {
        "average": avg_psi_gamma,
        "direct_effect": direct_effect,
        "spillover_effect": spillover_effect,
        "psi_1_gamma": psi_1_gamma,
        "psi_0_gamma": psi_0_gamma,
        "psi_zero": psi_zero,
    }

doubly_robust_optimized_debug(A_chain[-1], L_chain[-1], Y_chain[-1], adj)

[2.00776731 2.76346505 2.76901912 2.27685091 2.93332973 2.88434615
 3.17410733 3.0564813  2.33675073 2.29967421 2.96666132 4.70477878
 4.46501719 2.11032003 2.11359901 2.09580888 2.21655518 3.17602906
 2.76517791 3.92189041 2.26742055 2.25793069 2.03919625 2.18647255
 3.10544468 2.71561201 4.42362918 2.06885728 2.02863728 3.03523457
 1.92678242 2.22622765 3.06224937 2.7609939  2.67533606 3.02106674
 2.13097052 3.31612629 2.04671669 2.18451655 1.98587915 2.18314757
 2.66974176 1.93863547 2.17674661 3.06612764 2.24959643 2.98904133
 4.40006458 2.12940061 2.06418382 2.65930578 1.93863547 3.13143522
 3.18569219 2.35660795 2.27795953 3.01302729 4.39885832 2.10794522
 3.07814378 2.00651298 2.0536207  4.01755691 3.02359874 2.67721363
 2.33675073 2.98089408 2.02101112 2.1720401  2.13955396 2.98089408
 4.38138953 2.97910338 2.98899211 2.35253006 2.07167746 2.79827196
 3.0203904  2.78050783 1.98720054 3.18982673 2.25565514 3.01760173
 3.06314903 2.65931095 2.19396241 2.20249144 2.2162998  2.2748

{'average': 0.3167795763881033,
 'direct_effect': -0.19484655570771792,
 'spillover_effect': -0.020862063223996585,
 'psi_1_gamma': 0.2626460676197822,
 'psi_0_gamma': 0.4574926233275001,
 'psi_zero': 0.4783546865514967}